In [1]:
import csv
import xlrd
import numpy as np
import os

In [ ]:
datafile = '2013_ERCOT_Hourly_Load_Data.xls'

In [ ]:
def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    
    data = [[sheet.cell_value(r,col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]
    
    print('\nList Comprehension')
    print('data[4][3]:')
    print(data[4][3])
    
    print('\nCells in a nested loop:')
    for row in range(sheet.nrows):
        for col in range(sheet.ncols):
            if row == 50:
                print(sheet.cell_value(row,col))
                
    print("\nROWS, COLUMNS, and CELLS:")
    print( "Number of rows in the sheet:" )
    print( sheet.nrows)
    print ("Type of data in cell (row 3, col 2):")
    print( sheet.cell_type(3, 2))
    print( "Value in cell (row 3, col 2):" )
    print( sheet.cell_value(3, 2))
    print( "Get a slice of values in column 3, from rows 1-3:")
    print( sheet.col_values(3, start_rowx=1, end_rowx=4))

    print( "\nDATES:")
    print( "Type of data in cell (row 1, col 0):")
    print( sheet.cell_type(1, 0))
    exceltime = sheet.cell_value(1, 0)
    print( "Time in Excel format:")
    print( exceltime)
    print( "Convert time to a Python datetime tuple, from the Excel float:",)
    print( xlrd.xldate_as_tuple(exceltime, 0))

    return data

data = parse_file(datafile)
                     

In [ ]:
import xlrd
from zipfile import ZipFile
datafile = "2013_ERCOT_Hourly_Load_Data.xls"


def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    coast_data = sheet.col_values(1, 1)
    max_load = max(coast_data)
    max_index = coast_data.index(max_load)
    max_time = sheet.cell_value(max_index + 1, 0)
    max_time = xlrd.xldate_as_tuple(max_time, 0)
    min_load = min(coast_data)
    min_index = coast_data.index(min_load)
    min_time = sheet.cell_value(min_index + 1, 0)
    min_time = xlrd.xldate_as_tuple(min_time, 0)
    average_load = sum(coast_data)/len(coast_data)
    ### example on how you can get the data
    #sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]

    ### other useful methods:
    # print "\nROWS, COLUMNS, and CELLS:"
    # print "Number of rows in the sheet:", 
    # print sheet.nrows
    # print "Type of data in cell (row 3, col 2):", 
    # print sheet.cell_type(3, 2)
    # print "Value in cell (row 3, col 2):", 
    # print sheet.cell_value(3, 2)
    # print "Get a slice of values in column 3, from rows 1-3:"
    # print sheet.col_values(3, start_rowx=1, end_rowx=4)

    # print "\nDATES:"
    # print "Type of data in cell (row 1, col 0):", 
    # print sheet.cell_type(1, 0)
    # exceltime = sheet.cell_value(1, 0)
    # print "Time in Excel format:",
    # print exceltime
    # print "Convert time to a Python datetime tuple, from the Excel float:",
    # print xlrd.xldate_as_tuple(exceltime, 0)
    
    
    data = {
            'maxtime': (0, 0, 0, 0, 0, 0),
            'maxvalue': 0,
            'mintime': (0, 0, 0, 0, 0, 0),
            'minvalue': 0,
            'avgcoast': 0
    }
    data['maxtime'] = max_time
    data['maxvalue'] = round(max_load, 2)
    data['mintime'] = min_time
    data['minvalue'] = round(min_load, 2)
    data['avgcoast'] = round(average_load, 2)
    return data

print(parse_file(datafile) )


In [ ]:
import json
import requests


BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"

# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    # This is the main function for making queries to the musicbrainz API.
    # A json document should be returned by the query.
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print( "requesting", r.url)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    # This adds an artist name to the query parameters before making
    # an API call to the function above.
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    # After we get our output, we can format it to be more readable
    # by using this function.
    if type(data) == dict:
        print( json.dumps(data, indent=indent, sort_keys=True))
    else:
        print(data)


def main():
    '''
    Modify the function calls and indexing below to answer the questions on
    the next quiz. HINT: Note how the output we get from the site is a
    multi-level JSON document, so try making print statements to step through
    the structure one level at a time or copy the output to a separate output
    file.
    '''
    results = query_by_name(ARTIST_URL, query_type["simple"], "First Aid Kit")
    artists = results['artists']
    counter = 0
    for band in artists:
        
        if band['name'] == 'First Aid Kit':
            print(band)
            counter += 1
    print(counter)

     




    # pretty_print(results)

    artist_id = results["artists"][1]["id"]
    # print( "\nARTIST:")
    #pretty_print(results["artists"][1])

    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]
    # print( "\nONE RELEASE:")
    #pretty_print(releases[0], indent=2)
    release_titles = [r["title"] for r in releases]
    

    # print( "\nALL TITLES:")
    for t in release_titles:
        print(t)


if __name__ == '__main__':
    main()


In [ ]:
datafile = '745090.csv'

def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'r') as f:
        r = csv.reader(f, delimiter=",")
        station = next(r)[1]
        next(r)
        for row in r:
            data.append(row)
        
    # Do not change the line below
    name = station
    return (name, data)

parse_file(datafile)


In [43]:
datafile = '2013_ERCOT_Hourly_Load_Data.xls'

def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    station_dict = {}
    for i in range(1, sheet.ncols-1):
        
        # Station names are in row 0, starting at column 1
        station_name = sheet.cell_value(0, i)

        # Retrieve the data, starts in column 1 row 1
        # sheet.col_values(column, starting_row, ending_row=None)
        data = sheet.col_values(i, 1)
        
        # Retrieve maximum load in the column (station)
        max_load = max(data)
        
        # Retrieve index of maximum load (time)
        max_index = data.index(max_load)
        
        # Retrieve time of maximum load (increase row by 1), convert time to tuple
        # sheet.cell_value(row, column)
        max_time = sheet.cell_value(max_index+1, 0)
        max_time = xlrd.xldate_as_tuple(max_time, 0)
        
        max_year = max_time[0]
        max_month = max_time[1]
        max_day = max_time[2]
        max_hour = max_time[3]
        
        station_dict[station_name] = ([max_year, max_month, max_day, \
                                                    max_hour, max_load])
        
    return station_dict
    
station_dict = parse_file(datafile)
    
print(station_dict)     
    

{'COAST': [2013, 8, 13, 17, 18779.025510000003], 'EAST': [2013, 8, 5, 17, 2380.1654089999956], 'FAR_WEST': [2013, 6, 26, 17, 2281.2722140000024], 'NORTH': [2013, 8, 7, 17, 1544.7707140000005], 'NORTH_C': [2013, 8, 7, 18, 24415.570226999993], 'SOUTHERN': [2013, 8, 8, 16, 5494.157645], 'SOUTH_C': [2013, 8, 8, 18, 11433.30491600001], 'WEST': [2013, 8, 7, 17, 1862.6137649999998]}


In [44]:
out_file = '2013_Max_Loads.csv'

def save_file(out_file):
    with open(out_file, 'w') as f:
        w = csv.writer(f, delimiter="|")
        w.writerow(['Station', 'Year' , 'Month' , 'Day', 'Hour', 'Max Load'])
        for station, entry in station_dict.items():
            station_data = [station] + entry
            print(station_data)
            w.writerow(station_data)
        
save_file(out_file)

['COAST', 2013, 8, 13, 17, 18779.025510000003]
['EAST', 2013, 8, 5, 17, 2380.1654089999956]
['FAR_WEST', 2013, 6, 26, 17, 2281.2722140000024]
['NORTH', 2013, 8, 7, 17, 1544.7707140000005]
['NORTH_C', 2013, 8, 7, 18, 24415.570226999993]
['SOUTHERN', 2013, 8, 8, 16, 5494.157645]
['SOUTH_C', 2013, 8, 8, 18, 11433.30491600001]
['WEST', 2013, 8, 7, 17, 1862.6137649999998]
